# Large-Scale Image Processing On Your Laptop (and elsewhere)

### ImageXD 2017 Tutorial

---
### Goals of this workshop:

* get acquanted with the Python [Dask](http://dask.pydata.org/en/latest/) Library
* learn how to execute basic operations on large images which cannot fit in RAM
* learn about the concepts of lazy evaluation and task scheduling graphs
* learn how to work with [Dask Arrays](http://dask.pydata.org/en/latest/array.html)
* learn how to work with [Dask Delayed](http://dask.pydata.org/en/latest/delayed.html)

---
### Motivation

What is this tutorial about? Researchers across domains get overloaded with image data which their traditional processing workflows are incapable to handle. Usually they are faced with two possible options: 
* move the processing to large machines/clusters
* modify their methods to access the image data only pieces at a time.

<!---* Mechanical Engineering - microbubbles
* Oceanography - sonar, video
* Neuroscience - calcium imaging, fMRI
* etc--->


Scientists like to test out things on their laptops, and later move to clusters, without having to modify their code a lot.

[Dask](http://dask.pydata.org/en/latest/) is a Python Library which makes this possible: 
* can perform computations on images which cannot fit into RAM
* has interface similar to `numpy` and `scipy`
* the same code used on your laptop can be run on a distributed cluster


The Dask library provides several useful objects
* [Dask Arrays](http://dask.pydata.org/en/latest/array.html) (for numpy-like operations) *
* [Dask DataFrames](http://dask.pydata.org/en/doc-test-build/dataframe.html) (for pandas-like operations)
* [Dask Bags](http://dask.pydata.org/en/latest/bag.html) (for unstructured-data-like operations, such as text)
* [Dask Delayed](http://dask.pydata.org/en/latest/delayed.html) (for general functions and operations) *

We will be working with the [cells data](https://github.com/imagexd/imagexd.github.io/blob/master/assets/cells.tar.gz). So set the path to the location where you have unzipped the cells folder.

In [ ]:
path = '../data/cells/'

In [ ]:
# some preliminary imports
import numpy as np
%matplotlib inline
import matplotlib
import matplotlib.pyplot as plt

### Dask Arrays

Dask arrays are similar to numpy arrays, except that they are chunked into small pieces.

In [ ]:
import dask.array as da
from dask.array.image import imread

We can read a collection of images into a dask array.

In [ ]:
cell_images = imread(path+"*.jpg")

Let's look at it:

In [ ]:
cell_images

In [ ]:
# saving the dimensions
T, m, n, d = cell_images.shape

Each image is a separate chunk.

*The images are not loaded into RAM!!!!*

We can do computations on the dask array as long as individual chunks (and the computations on them) fit into RAM.

Let's  find the mean image:

In [ ]:
cell_images.mean(axis=0)

The above statement creates another dask array, which is not loaded into RAM, i.e. it is not calculated yet. 
It waits for us tell dask explicitly when to do the calculation using the `compute` command.This is called [lazy evaluation](https://en.wikipedia.org/wiki/Lazy_evaluation).

In [ ]:
%%time
mean_cell_image = cell_images.mean(axis=0).compute()

In [ ]:
type(mean_cell_image)

In [ ]:
plt.imshow(mean_cell_image)

Well, that is pretty but let's do something more meaningful!

***Important: only the final image is stored in RAM, which is smaller than the original collection!***

Side Note: if we pass a dask array to a function not supporting dask arrays, it often gets converted to an in-memory numpy array and evaluated.

A lot of other functions to perform on images using the [Dask API](http://dask.pydata.org/en/latest/array-api.html).

#### Exercise:
* calculate the mean for each RGB color for each image
* calculate the max for each RGB color for each image
* calculate the min and max across all colors for each image

In [ ]:
# mean for each RGB color for each image
RGB_means = cell_images.mean(axis=[1,2])
plt.plot(RGB_means,'o')

In [ ]:
# max for each color for each image

In [ ]:
# min across all colors for each image

In [ ]:
# max across all colors for each image

We observe that some of the images have a very diffent intensity range.

#### A simple cell classifier

We can try to use the means of the colors to distinguish between images of differently colored cells.
Let's find the images which have higher green color than red color.

In [ ]:
#average green vs average red 
green_labels = (RGB_means[:,0]<RGB_means[:,1]).compute()

In [ ]:
plt.figure()
plt.imshow(cell_images[green_labels,:][3,:,:,:])
plt.show()

Key Points:

* dask arrays are not loaded into memory until you start doing computations on them
* computations on dask arrays are not executed until you ask for them via `.compute`()
* passing a `dask.array` to a `numpy` or `matplotlib` function might work but the computation will be done in memory

---
### Perform Principal Component Analysis on a Collection of Images

PCA can be perform by the following steps:
* center the images
* reshape the data into features x observations format
* perform SVD on the data matrix
* reshape the modes into an image format -> they represent the principal components

In [ ]:
# center the images
cell_images_centered = cell_images - cell_images.mean(axis = 0)

In [ ]:
cell_images_centered

In [ ]:
# create a data matrix - # features x #observations
data = cell_images_centered.reshape((T,m*n*d)).T
print(data.shape)
data

In [ ]:
# svd expects only one column block
data = data.rechunk((256,T))
data

In [ ]:
u, s, vt = da.linalg.svd(data)

In [ ]:
u = u[:,:10].compute()

In [ ]:
vt = vt[:10,:]

Recal that principal components are unique up to a sign change. So in order that we all obtain the same result we can flip the sings based on vt using the following `svd_flip` function.

In [ ]:
# flip signs based on vt
from sklearn.utils.extmath import svd_flip
u, vt = svd_flip(u,vt)

In [ ]:
# convert the components to back to images
u = u.reshape(256,256,3,10)

In [ ]:
plt.figure(figsize = (10,5))
for i in range(10):
    plt.subplot(2,5,i+1)
    # we are rescaling between 0 and 1 before plotting
    plt.imshow((u[:,:,:,i]-u.min())/(u.max() - u.min()))
    plt.title('Mode '+str(i+1))

Key Points:
* some dask functions are strict about the form of the chunks
* not all familiar methods are implemented within dask, but you can still run them using the existing base functions

#### Extra Exercises

**Exercise: chunking performance**

Try using different chunk sizes and compare the speed of the svd computation.

**Exercise: faster svd**

If you compare to a `scipy` or `scikit-learn` implementations you will notice that the dask svd is a bit slow. The reason is that when the data is too big the existing packages by default do not compute the full `svd` , instead they implement a randomized svd algorithm. Dask has the corresponding implementation in `da.linalg.svd_compressed`.

In [ ]:
help(da.linalg.svd_compressed)

Run the compressed svd algorithm and compare the execution times.

----
### Performing complex operations on a stack of images.

We consider the following scenario: 

we have a big pile of images and we need to perform the same pre-processing step to each image and in the end store the result in one array for further processing.

Clearly we can achieve this by writing a `for-loop` which processes each image and stores the result. 

To accomplish this in a distributed manner we can use `dask`'s [delayed](http://dask.pydata.org/en/latest/delayed.html) functionality. It allows to parallelize our own Python functions.

In [ ]:
from dask import delayed

We recall that the images had different intensity ranges: let's rescale them so that they are always between 0 and 255. We can achieve that with the scikit-image [exposure.rescale_intensity](http://scikit-image.org/docs/dev/api/skimage.exposure.html#skimage.exposure.rescale_intensity) funcion.

In [ ]:
from skimage.exposure import rescale_intensity

We can convert a regular Python function to a dask delayed function:

In [ ]:
rescale_intensity_dask = delayed(rescale_intensity)

In [ ]:
# create delayed evaluations
lazy_values = [rescale_intensity_dask(im) for im in cell_images]

In [ ]:
# create a list of dask arrays
arrays = [da.from_delayed(lazy_value, shape=(m,n,d), dtype=cell_images.dtype) for lazy_value in lazy_values]

In [ ]:
type(arrays[0])

We can see that the type of this final result is a list and each entry is a `dask array`:

In [ ]:
print(type(arrays))
print(type(arrays[0]))

We can convert them to a dask array:

In [ ]:
stack = da.stack(arrays, axis=0) 

In [ ]:
stack

Note: this chunksize is useful when doing individual operations on images, not across images!

In [ ]:
plt.subplot(1,2,1)
plt.imshow(cell_images[100,:,:,:])
plt.title('Original Image')
plt.subplot(1,2,2)
plt.imshow(stack[100,:,:,:])
plt.title('Rescaled Image')

In [ ]:
RGB_max = stack.max(axis = [1,2,3]).compute()

Now the maximum is always 255.

In [ ]:
plt.plot(RGB_max,'ko')

Note: there are alternative approaches to applying a function to a collection of objects in a distributed manner, `dask delayed` is most useful when we want to store the results in a distributed array on which to perform further operations.

Key Points:
* any Python function can be converted to a dask function using `delayed`
* it is most useful to use `dask delayed` when the output can be stored in a dask object
* the power of `dask delayed` is more noticeable on many-core machines or clusters

#### Extra Exercises

**Exercise: segmenting images**

One approach to segment color images is using the k-means algorithm in color space. The `scikit-image` [`segmentation.slic`](http://scikit-image.org/docs/dev/api/skimage.segmentation.html#skimage.segmentation.slic) function is a good candidate:

In [ ]:
from skimage import segmentation
help(segmentation.slic)

Apply the function to each image in the collection using `dask delayed`.

Hint: first test the segmentation on a single image to select appropriate parameters.

Compare the speed of this approach to applying the algorithm through a for loop.

#### Exercise: calculating histograms

Use the same approach as above to calculate histogram for each image. In the new array each image is represented by its histogram. You can further try a clustering algorithm to cluster the images based on their histograms.

----
### Further Resources
* [Scipy 2016 Parallel Python Tutorial](https://www.youtube.com/watch?v=5Md_sSsN51k&list=PLYx7XA2nY5Gf37zYZMw6OqGFRPjB1jCy6&index=13) - comparison of different approaches for parallelization in Python
* [Distributed Images Blog](http://matthewrocklin.com/blog/work/2017/01/17/dask-images) - an example of processing a large image dataset on an Amazon EC2 cluster 
* [Dask on an Amazon EC2 cluster](https://github.com/dask/dask-ec2) - tool to start up a dask EC2 cluster